In [13]:
import os
import subprocess
import pandas as pd
import numpy as np
import json
from datetime import datetime

import requests
from bs4 import BeautifulSoup

from scipy import optimize
from scipy import integrate
from scipy import signal

from sklearn import linear_model
reg = linear_model.LinearRegression(fit_intercept=True)

pd.set_option('display.max_rows', 500)

# Source : John Hopkins Dataset
# Data understanding and preparation

In [4]:
#Get latest data from JH and store in csv
def store_relational_JH_data():
    ''' Transformes the COVID data in a relational data set

    '''

    data_path='../data/raw/COVID-19/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_global.csv'
    pd_raw=pd.read_csv(data_path)

    pd_data_base=pd_raw.rename(columns={'Country/Region':'country',
                      'Province/State':'state'})

    pd_data_base['state']=pd_data_base['state'].fillna('no')

    pd_data_base=pd_data_base.drop(['Lat','Long'],axis=1)


    pd_relational_model=pd_data_base.set_index(['state','country']) \
                                .T                              \
                                .stack(level=[0,1])             \
                                .reset_index()                  \
                                .rename(columns={'level_0':'date',
                                                   0:'confirmed'},
                                                  )

    pd_relational_model['date']=pd_relational_model.date.astype('datetime64[ns]')

    pd_relational_model.to_csv('../data/processed/COVID_relational_confirmed.csv',sep=';',index=False)
    print(' Number of rows stored: '+str(pd_relational_model.shape[0]))
    print(' Latest date is: '+str(max(pd_relational_model.date)))
if __name__ == '__main__':

    store_relational_JH_data()


 Number of rows stored: 63042
 Latest date is: 2020-09-14 00:00:00


In [5]:
# Get the data from the URL mentioned above
def getLatestDataFromURL(info_type):
    if info_type == "confirmed":
        response = requests.get("https://github.com/CSSEGISandData/COVID-19/blob/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_global.csv")
    elif info_type == "deaths":
        response = requests.get("https://github.com/CSSEGISandData/COVID-19/blob/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_deaths_global.csv")
    elif info_type == "recovered":
        response = requests.get("https://github.com/CSSEGISandData/COVID-19/blob/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_recovered_global.csv")
    soup = BeautifulSoup(response.content, 'html.parser')
    html_table=soup.find('table')
    all_rows=html_table.find_all('tr')
    jh_data_list=[]
    for pos,rows in enumerate(all_rows):
        if pos==0:
            header_list = [each_col.get_text(strip=True) for each_col in rows.find_all('th')]
        else:
            col_list=[each_col.get_text(strip=True) for each_col in rows.find_all('td')] #td for data element
            jh_data_list.append(col_list)
    return jh_data_list,header_list
    

In [6]:
# Prepare the data for visualization and modelling
def prepareData(jh_data_list,header_list):
    header_list.insert(0,'index')
    jh_data_df=pd.DataFrame(jh_data_list)
    jh_data_df.columns=header_list
    #jh_data_df.head()
    time_idx=jh_data_df.columns[5:]
    country_list=jh_data_df['Country/Region']
    jh_data_transformed_df = pd.DataFrame({'date':time_idx})
    for each in country_list:
        jh_data_transformed_df[each] = np.array(jh_data_df[jh_data_df['Country/Region']==each].iloc[:,5::].astype(int).sum(axis=0))
    #jh_data_transformed_df.tail()
    time_idx=[datetime.strptime( each,"%m/%d/%y") for each in jh_data_transformed_df.date] # convert to datetime
    time_str=[each.strftime('%Y-%m-%d') for each in time_idx] # convert back to date ISO norm (str)
    jh_data_transformed_df['date']=time_idx
    return jh_data_transformed_df

In [9]:
confirmed_list,header_list = getLatestDataFromURL("confirmed")
confirmed_df = prepareData(confirmed_list,header_list)
confirmed_df.to_csv('../data/processed/COVID_small_flat_table_confirmed.csv',sep=';',index=False)
deaths_list,header_list = getLatestDataFromURL("deaths")
deaths_df = prepareData(deaths_list,header_list)
deaths_df.to_csv('../data/processed/COVID_small_flat_table_deaths.csv',sep=';',index=False)
recovered_list,header_list = getLatestDataFromURL("recovered")
recovered_df = prepareData(recovered_list,header_list)
recovered_df.to_csv('../data/processed/COVID_small_flat_table_recovered.csv',sep=';',index=False)

In [10]:
def getPopulationDataFromWM():
    response = requests.get("https://www.worldometers.info/world-population/population-by-country/")
    response.content
    soup = BeautifulSoup(response.content, 'html.parser')
    html_table=soup.find('table')
    all_rows=html_table.find_all('tr')
    jh_data_list=[]
    for pos,rows in enumerate(all_rows):
        if pos==0:
            header_list = [each_col.get_text(strip=True) for each_col in rows.find_all('th')]
        else:
            col_list=[each_col.get_text(strip=True) for each_col in rows.find_all('td')]
            jh_data_list.append(col_list)
    jh_data_df = pd.DataFrame(jh_data_list)
    return jh_data_df.iloc[:, 1:3]

In [11]:
pop_data = getPopulationDataFromWM()
countries = pop_data[1]
population = pop_data[2]
pop_df = pd.DataFrame(columns=countries)
pop_df.loc[len(pop_df)] = population.tolist()
pop_df.rename(columns={'United States': 'US'}, inplace=True)
pop_df.to_csv('../data/processed/population.csv',sep=';',index=False)

# Doubling rate and filtering

In [14]:
def get_doubling_time_via_regression(in_array):
    ''' Use a linear regression to approximate the doubling rate

        Parameters:
        ----------
        in_array : pandas.series

        Returns:
        ----------
        Doubling rate: double
    '''

    y = np.array(in_array)
    X = np.arange(-1,2).reshape(-1, 1)

    assert len(in_array)==3
    reg.fit(X,y)
    intercept=reg.intercept_
    slope=reg.coef_

    return intercept/slope


def savgol_filter(df_input,column='confirmed',window=5):
    ''' Savgol Filter which can be used in groupby apply function (data structure kept)

        parameters:
        ----------
        df_input : pandas.series
        column : str
        window : int
            used data points to calculate the filter result

        Returns:
        ----------
        df_result: pd.DataFrame
            the index of the df_input has to be preserved in result
    '''

    degree=1
    df_result=df_input

    filter_in=df_input[column].fillna(0) # attention with the neutral element here

    result=signal.savgol_filter(np.array(filter_in),
                           window, # window size used for filtering
                           1)
    df_result[str(column+'_filtered')]=result
    return df_result

def rolling_reg(df_input,col='confirmed'):
    ''' Rolling Regression to approximate the doubling time'

        Parameters:
        ----------
        df_input: pd.DataFrame
        col: str
            defines the used column
        Returns:
        ----------
        result: pd.DataFrame
    '''
    days_back=3
    result=df_input[col].rolling(
                window=days_back,
                min_periods=days_back).apply(get_doubling_time_via_regression,raw=False)



    return result




def calc_filtered_data(df_input,filter_on='confirmed'):
    '''  Calculate savgol filter and return merged data frame

        Parameters:
        ----------
        df_input: pd.DataFrame
        filter_on: str
            defines the used column
        Returns:
        ----------
        df_output: pd.DataFrame
            the result will be joined as a new column on the input data frame
    '''

    must_contain=set(['state','country',filter_on])
    assert must_contain.issubset(set(df_input.columns)), ' Erro in calc_filtered_data not all columns in data frame'

    df_output=df_input.copy() # we need a copy here otherwise the filter_on column will be overwritten

    pd_filtered_result=df_output[['state','country',filter_on]].groupby(['state','country']).apply(savgol_filter)#.reset_index()

    #print('--+++ after group by apply')
    #print(pd_filtered_result[pd_filtered_result['country']=='Germany'].tail())

    #df_output=pd.merge(df_output,pd_filtered_result[['index',str(filter_on+'_filtered')]],on=['index'],how='left')
    df_output=pd.merge(df_output,pd_filtered_result[[str(filter_on+'_filtered')]],left_index=True,right_index=True,how='left')
    #print(df_output[df_output['country']=='Germany'].tail())
    return df_output.copy()





def calc_doubling_rate(df_input,filter_on='confirmed'):
    ''' Calculate approximated doubling rate and return merged data frame

        Parameters:
        ----------
        df_input: pd.DataFrame
        filter_on: str
            defines the used column
        Returns:
        ----------
        df_output: pd.DataFrame
            the result will be joined as a new column on the input data frame
    '''

    must_contain=set(['state','country',filter_on])
    assert must_contain.issubset(set(df_input.columns)), ' Erro in calc_filtered_data not all columns in data frame'


    pd_DR_result= df_input.groupby(['state','country']).apply(rolling_reg,filter_on).reset_index()

    pd_DR_result=pd_DR_result.rename(columns={filter_on:filter_on+'_DR',
                             'level_2':'index'})

    #we do the merge on the index of our big table and on the index column after groupby
    df_output=pd.merge(df_input,pd_DR_result[['index',str(filter_on+'_DR')]],left_index=True,right_on=['index'],how='left')
    df_output=df_output.drop(columns=['index'])


    return df_output


if __name__ == '__main__':
    test_data_reg=np.array([2,4,6])
    result=get_doubling_time_via_regression(test_data_reg)
    print('the test slope is: '+str(result))

    pd_JH_data=pd.read_csv('../data/processed/COVID_relational_confirmed.csv',sep=';',parse_dates=[0])
    pd_JH_data=pd_JH_data.sort_values('date',ascending=True).copy()

    #test_structure=pd_JH_data[((pd_JH_data['country']=='US')|
    #                  (pd_JH_data['country']=='Germany'))]

    pd_result_larg=calc_filtered_data(pd_JH_data)
    pd_result_larg=calc_doubling_rate(pd_result_larg)
    pd_result_larg=calc_doubling_rate(pd_result_larg,'confirmed_filtered')


    mask=pd_result_larg['confirmed']>100
    pd_result_larg['confirmed_filtered_DR']=pd_result_larg['confirmed_filtered_DR'].where(mask, other=np.NaN)
    pd_result_larg.to_csv('../data/processed/COVID_final_set.csv',sep=';',index=False)
    print(pd_result_larg[pd_result_larg['country']=='US'].tail())


the test slope is: [2.]
            date state country  confirmed  confirmed_filtered  confirmed_DR  \
59956 2020-09-10    no      US  6396100.0           6402419.2    184.137066   
59957 2020-09-11    no      US  6443652.0           6440932.0    153.403356   
59958 2020-09-12    no      US  6485123.0           6479620.0    144.718219   
59959 2020-09-13    no      US  6519573.0           6518722.5    170.777062   
59960 2020-09-14    no      US  6553652.0           6557825.0    190.268334   

       confirmed_filtered_DR  
59956             180.980210  
59957             169.810423  
59958             166.863307  
59959             166.595103  
59960             166.708586  
